# MultiAgentRL Colab Runner

이 노트북은 Colab에서 버전 충돌을 줄이고 `runMARL.py`를 바로 실행하기 위한 셀 구성입니다.
- 권장: `Runtime > Change runtime type > GPU`
- `runMARL.py`는 60 iteration으로 시간이 오래 걸릴 수 있습니다.

In [ ]:
REPO_URL = "https://github.com/jisang0706/MultiAgentRL_InventoryControl.git"
REPO_DIR = "/content/MultiAgentRL_InventoryControl"
TARGET_COMMIT = "1e44b26"  # optional check

import os

if os.path.exists(REPO_DIR):
    %cd {REPO_DIR}
    !git fetch origin
    !git checkout master
    !git reset --hard origin/master
else:
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}

!pwd
!git rev-parse --short HEAD
print("expect:", TARGET_COMMIT)
!ls -1 | head


In [2]:
# Core deps (Colab 충돌 최소화)
!pip -q install -U pip setuptools wheel
!pip -q install "jedi>=0.19.1"
!pip -q install "ray[rllib]==2.31.0" "omegaconf==2.3.0" "json5==0.9.25"
# Colab 기본 패키지 충돌 회피를 위해 numpy/scipy/pandas/matplotlib/tqdm는 고정하지 않음


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
inequality 1.1.2 requires scipy>=1.12, but you have scipy 1.11.4 which is incompatible.
giddy 2.3.8 requires scipy>=1.12, but you have scipy 1.11.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
access 1.1.10.post3 requires scipy>=1.14.1, but you have scipy 1.11.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires scipy>=1.13, but you have scipy 1.11.4 which is incompatible.
tobler 0.13.0 requires tqdm>=4.67, but you have tqdm 4.66.5 which is incompatible.
libpysal 4.14.1 requires scip

In [ ]:
# torch-geometric 호환 설치 (torch 버전에 맞춰 wheel URL 계산)
import subprocess, sys
import torch

torch_v = torch.__version__.split("+")[0]
cuda = torch.version.cuda
tag = f"cu{cuda.replace('.', '')}" if cuda else "cpu"
wheel = f"https://data.pyg.org/whl/torch-{torch_v}+{tag}.html"

print("torch:", torch.__version__)
print("pyg wheel:", wheel)
pkgs = ["pyg_lib", "torch_scatter", "torch_sparse", "torch_cluster", "torch_spline_conv"]
cmd = [sys.executable, "-m", "pip", "install", "-q", *pkgs, "-f", wheel]
rc = subprocess.run(cmd).returncode
if rc != 0:
    print("compiled wheel install failed; continue with base torch-geometric install")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch-geometric==2.5.3"])

In [ ]:
import sys, ray, gymnasium, numpy, torch
print("python:", sys.version)
print("ray:", ray.__version__)
print("gymnasium:", gymnasium.__version__)
print("numpy:", numpy.__version__)
print("torch:", torch.__version__)

In [ ]:
# Environment smoke test
from env3rundiv import MultiAgentInvManagementDiv

cfg = {
    "connections": {0: [1, 2], 1: [3, 4], 2: [4, 5], 3: [], 4: [], 5: []},
    "num_nodes": 6,
}
env = MultiAgentInvManagementDiv(cfg)
obs, infos = env.reset()
print("num agents:", len(obs))
print("agent ids sample:", list(obs.keys())[:3])
print("obs shape:", env.observation_space.shape, "act shape:", env.action_space.shape)

In [ ]:
# Ray compatibility patch for runMARL.py
from pathlib import Path
f = Path('runMARL.py')
txt = f.read_text()

needle_import = 'from ray.rllib.algorithms.qmix import QMixConfig\n'
if needle_import in txt:
    txt = txt.replace(needle_import, '')
    print('patched: removed qmix import')
else:
    print('qmix import: already patched')

old_env = 'env_config={\n            "num_agents": num_agents,\n        },'
new_env = 'env_config={\n            "connections": config["connections"],\n            "num_nodes": num_nodes,\n            "num_agents": num_agents,\n        },'
if old_env in txt:
    txt = txt.replace(old_env, new_env)
    print('patched: env_config now includes num_nodes/connections')
else:
    print('env_config: already patched or different format')

f.write_text(txt)


In [ ]:
# Main training (central critic)
!python runMARL.py

In [ ]:
# Optional: GNN variants
# !python rungnn.py
# !python rungnnpool.py